# 7.2 Data Transformation

1. [Removing Duplicates](#rmdup)
1. [Transforming Data Using a Function or Mapping](#mapping)
1. [Replacing Values](#replace)
1. [Renaming Axis Indexes](#rename)
1. [Discretization and Binning](#binning)
1. [Detecting and Filtering Outliers](#outlier)
1. [Permutation and Random Sampling](#permutation)
1. [Computing Indicator/Dummy Variables](#dummy)

<a name="rmdup"></a>
# Removing Duplicates

Two main methods for removing duplicates

1. Creating a boolean Series with the `duplicated` method and using that to subset
1. Using the `drop_duplicates` method to directly remove them

`drop_duplicates` is a little more robust because you can provide various arguments to it to modify the behavior

1. `subset` is a scalar or list of columns to use to consider what are duplicates (default is to consider all columns)
1. `keep` determines which duplicate to keep. `first` is default, `last` is available as well

In [109]:
import pandas as pd
import numpy as np

data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [110]:
# Boolean array is returned
# Note how first occurrence is False b/c it's not duplicated yet
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [111]:
# Subset with boolean array
data[~data.duplicated()]

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [112]:
# Subset with drop_duplicates
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [113]:
# Add another column
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [114]:
# Default doesn't do anything now 
data.drop_duplicates()

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [115]:
# Use only k1 to drop
data.drop_duplicates(subset="k1")

,k1,k2,v1
0,one,1,0
1,two,1,1


In [116]:
# Use both and also keep the last occurrence, not the first
data.drop_duplicates(subset=["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


<a name="mapping"></a>
# Transforming Data Using a Function or Mapping

This is something that gives me a bit of troulbe in R, so pay attention!

Basically, how to map a smaller set of values to a larger set.

In the example below, we have a DataFrame with different meats and a measurement. The relationship of `meat type` to `animal` is `many-to-one`, so how do we map all the different meats that come from a cow to a new animal column?

Use map! (Also found in Ch05.2)

In [117]:
# DataFrame of different meats and their measurement of some sort in ounces
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [118]:
# Dictionary assigning each meat to an animal
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

In [119]:
# Use `map` method and mapping dict to create new column
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [120]:
# Alternatively, make a re-usable mapping function and then apply
def get_animal(x):
    """
    Given a key from meat_to_animal dict, return value
    """
    return meat_to_animal[x]

In [121]:
# Apply function
data["animal2"] = data["food"].map(get_animal)
data

,food,ounces,animal,animal2
0,bacon,4.0,pig,pig
1,pulled pork,3.0,pig,pig
2,bacon,12.0,pig,pig
3,pastrami,6.0,cow,cow
4,corned beef,7.5,cow,cow
5,bacon,8.0,pig,pig
6,pastrami,3.0,cow,cow
7,honey ham,5.0,pig,pig
8,nova lox,6.0,salmon,salmon


<a name="replace"></a>
# Replacing Values

Have already seen various ways to replace certain values (`fillna`, `map`, etc.).  

`replace` is a straightforward method to replace values.  

The general idea is `obj.replace(find, replace)`.  
1. "find" and "replace" can both be single values
1. "find" can be a list and "replace" a scalar to replace everything in find with one value
1. "find" and "replace" can be lists of equal length to do it element-wise
1. Can also provide a dictionary of key:value ~ find:replace instead in the format `obj.replace({key:val, key2:val2})`

In [122]:
# Series
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [123]:
# Replace a single value
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [124]:
# Replace multiple values with a single
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [125]:
# element-wise replacement of lists of values
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [126]:
# Dictionary instead
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

<a name="rename"></a>
# Renaming Axis Indexes

The axes of a DataFrame also have the `map` method.

Map can be used to modify in place. You can also use `replace` to transform an object without modifying the original

In [127]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [128]:
# Define a function
def cut4Upper(x):
    # Take first 4 elements of string and return in uppercase
    return x[:4].upper()

In [129]:
# Apply the function using map
data.index.map(cut4Upper)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [130]:
# Assign the applied function
data.index = data.index.map(cut4Upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [131]:
# Rename
data.rename(index=str.title,        # use the title method for strings to capitalize first letter only of row names
            columns=str.upper)      # use the upper method for strings to turn column names to all caps

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Rename also has the flexibility of taking dictionaries of find/replace for either axis. The keys have to be valid row/column names and then the values are what to replace it with

In [132]:
# Replace OHIO with INDIANA and three with peekaboo
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


<a name="binning"></a>
# Discretization and Binning

Bin discrete values (aka for histograms).  

The main method for accomplishing this is `pandas.cut`, which returns a special Categorical object

In [133]:
# Build a list of ages
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [134]:
# Create a list of bins.
# Cut handles this (first, second); (second+1, third), (third+1, fourth), etc
# e.g. 18-25; 26-35; 36-60; 60-100
# It's actually 19-25 (see below)
bins = [18, 25, 35, 60, 100]

In [135]:
# Bin the list with cut
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

Note the special characteristics of this new object:
1. Each value in the list is converted to its corresponding bin (aka can now be aggregated with `value_counts`)
1. The bin categories are also listed

The `codes` attribute further converts each list element into its bin index (e.g. 18-25 is 0, 26-35 is 1, etc.).  

The `categories` attribute simply contains the bin groupings.  

Remember that `(` means open/excluded and `]` means closed/included in these ranges.  

The `right` argument determines which side of the range is opened or closed.  

1. `right=True` (default) closes the upper value
1. `right=False` closes the lower value

In [136]:
# View the list values as their bin codes
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [137]:
# View the bin categories
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [138]:
# View the first bin
age_categories.categories[0]

Interval(18, 25, closed='right')

In [139]:
# View the counts of each bin
# alternative method: pd.value_counts(age_categories)
age_categories.value_counts()

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Let's add a value of 18 to the `ages` list so that we can see how the open/close works.

The default behavior will return `NaN` for 18 because it is currently out of our range, since `(18,25]` excludes 18 and includes 25.  

If we change the `right` argument to be `False`, we'll get a value returned. Also notice how the 3rd element (25) changes groups from the first bin to the second.

In [140]:
# Add our mininum age
ages.append(18)
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32, 18]

In [141]:
# Default cut returns NA
pd.cut(ages, bins)

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (60.0, 100.0], (35.0, 60.0], (35.0, 60.0], (25.0, 35.0], NaN]
Length: 13
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [142]:
# Changing the include/exclude side captures 18 and changes bin membership of 25
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [60, 100), [35, 60), [35, 60), [25, 35), [18, 25)]
Length: 13
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

The numeric (interval-based) labeling of the bins is kind of hard to read. A list or array of labels (must be same length as bin list!) can also be provided using the `labels` argument to make the output a bit clearer.

In [143]:
# Create a list of group names
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
group_names

['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [144]:
# Cut with those
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult', NaN]
Length: 13
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

We can create different types of bins instead of predefined groupings as well.  

1. Providing a single integer to `cut` will group things into equal-length bins based on the min and max
    - Note that this means the **range** covered by each bin will be equal, they will likely have different numbers of elements in them.
    - use `precision` argument to determine how many decimals to use to determine cut-off
1. Using `qcut` is very similar, but cuts using quantiles
    - Depending on the distribution, this will give differently-lengthed bin ranges, but each bin will have the same/similar numbers of element.
    - Instead of just 1 integer (i.e. 4 for quartiles), you can provide custom quantiles between 0 and 1 inclusive

In [145]:
# Make some data
data = np.random.uniform(size=20)
data

array([0.74771481, 0.96130674, 0.0083883 , 0.10644438, 0.29870371,
       0.65641118, 0.80981255, 0.87217591, 0.9646476 , 0.72368535,
       0.64247533, 0.71745362, 0.46759901, 0.32558468, 0.43964461,
       0.72968908, 0.99401459, 0.67687371, 0.79082252, 0.17091426])

In [146]:
# Cut into 4 groups - two decimal precision
myBins = pd.cut(data, 4, precision=2)
myBins

[(0.75, 0.99], (0.75, 0.99], (0.0074, 0.25], (0.0074, 0.25], (0.25, 0.5], ..., (0.5, 0.75], (0.75, 0.99], (0.5, 0.75], (0.75, 0.99], (0.0074, 0.25]]
Length: 20
Categories (4, interval[float64, right]): [(0.0074, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 0.99]]

In [147]:
# Cut into 4 groups - four decimal precision
pd.cut(data, 4, precision=4)

[(0.7476, 0.994], (0.7476, 0.994], (0.007403, 0.2548], (0.007403, 0.2548], (0.2548, 0.5012], ..., (0.5012, 0.7476], (0.7476, 0.994], (0.5012, 0.7476], (0.7476, 0.994], (0.007403, 0.2548]]
Length: 20
Categories (4, interval[float64, right]): [(0.007403, 0.2548] < (0.2548, 0.5012] < (0.5012, 0.7476] < (0.7476, 0.994]]

In [148]:
# Note the different sized bins
myBins.value_counts()

(0.0074, 0.25]    3
(0.25, 0.5]       4
(0.5, 0.75]       6
(0.75, 0.99]      7
Name: count, dtype: int64

In [149]:
# New data
data = np.random.standard_normal(1000)
data

array([ 3.24894392e+00, -1.02122752e+00, -5.77087303e-01,  1.24121276e-01,
        3.02613562e-01,  5.23772068e-01,  9.40277775e-04,  1.34380979e+00,
       -7.13543985e-01, -8.31153539e-01, -2.37023165e+00, -1.86076079e+00,
       -8.60757398e-01,  5.60145293e-01, -1.26593449e+00,  1.19827125e-01,
       -1.06351245e+00,  3.32882716e-01, -2.35941881e+00, -1.99542955e-01,
       -1.54199553e+00, -9.70735912e-01, -1.30703025e+00,  2.86349747e-01,
        3.77984111e-01, -7.53886535e-01,  3.31285650e-01,  1.34974221e+00,
        6.98766888e-02,  2.46674110e-01, -1.18616011e-02,  1.00481159e+00,
        1.32719461e+00, -9.19261558e-01, -1.54910644e+00,  2.21845987e-02,
        7.58363145e-01, -6.60524328e-01,  8.62580083e-01, -1.00319021e-02,
        5.00093559e-02,  6.70215594e-01,  8.52965032e-01, -9.55868852e-01,
       -2.34933207e-02, -2.30423388e+00, -6.52468841e-01, -1.21830198e+00,
       -1.33260971e+00,  1.07462269e+00,  7.23641505e-01,  6.90001853e-01,
        1.00154344e+00, -

In [150]:
# Split into quartiles
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(0.62, 3.93], (-2.96, -0.69], (-0.69, -0.019], (-0.019, 0.62], (-0.019, 0.62], ..., (0.62, 3.93], (-0.69, -0.019], (-0.69, -0.019], (-0.69, -0.019], (-0.69, -0.019]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.96, -0.69] < (-0.69, -0.019] < (-0.019, 0.62] < (0.62, 3.93]]

In [151]:
# Now we have even bins
quartiles.value_counts()

(-2.96, -0.69]     250
(-0.69, -0.019]    250
(-0.019, 0.62]     250
(0.62, 3.93]       250
Name: count, dtype: int64

In [152]:
# Custom quantiles
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-2.9499999999999997, -1.191]    100
(-1.191, -0.0192]                400
(-0.0192, 1.289]                 400
(1.289, 3.928]                   100
Name: count, dtype: int64

<a name="outlier"></a>
# Detecting and Filtering Outliers

Basically just specific array operations.

In [153]:
# Make some normally-distributed data
data = pd.DataFrame(np.random.standard_normal((1000, 4)))

In [154]:
# View it
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.022282,-0.001383,-0.055245,0.043350
std,1.008995,0.995882,0.998735,0.996658
min,-3.184377,-3.745356,-3.428254,-3.645860
25%,-0.628122,-0.697084,-0.747963,-0.600254
50%,-0.016127,-0.026381,-0.096965,0.043663
75%,0.695298,0.699046,0.622384,0.740562
max,3.525865,2.735527,3.366626,2.653656


In [155]:
# Find all values in column 2 that are greater than abs(3)
colData = data[2]
colData[colData.abs() > 3]

267   -3.428254
644    3.366626
Name: 2, dtype: float64

In [156]:
# Find all rows that have a value greater than abs(3) *in any column*
# Remember the () convention to allow methods to be applied to intermediate results
data[(data.abs() > 3).any(axis = "columns")]

,0,1,2,3
50,-0.025907,-3.399312,-0.974657,-0.685312
69,3.260383,0.963301,1.201206,-1.852001
145,-0.196713,-3.745356,-1.520113,-0.346839
244,-3.056990,1.918403,-0.578828,1.847446
267,0.326045,0.425384,-3.428254,-0.296336
331,-3.184377,1.369891,-1.074833,-0.089937
552,0.208011,-0.150923,-0.362528,-3.548824
644,0.193299,1.397822,3.366626,-2.372214
791,3.525865,0.283070,0.544635,0.462204
811,-0.450721,-0.080332,0.599947,-3.645860


In [157]:
# Convert all values outside of abs(3) to +3 or -3 accordingly
# Why doesn't the .any need to be used here? Have to remember what the comparison and what any both do.
# data[data.abs() > 3] returns an array where any value greater than abs(3) is returned as that value and everything else is returned as NA.
# The any method along with axis="columns" is used to search that DataFrame for rows where any column has an NA. Those are then returned above.
# For this though, we just need to assign a new value to them
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.021737,-0.000238,-0.055183,0.044545
std,1.005754,0.992064,0.996186,0.992637
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.628122,-0.697084,-0.747963,-0.600254
50%,-0.016127,-0.026381,-0.096965,0.043663
75%,0.695298,0.699046,0.622384,0.740562
max,3.000000,2.735527,3.000000,2.653656


In [158]:
# np.sign(data) returns the sign (i.e. -1 or +1) of each value
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,1.0,-1.0
1,1.0,1.0,-1.0,-1.0
2,1.0,-1.0,-1.0,-1.0
3,1.0,1.0,-1.0,1.0
4,-1.0,-1.0,-1.0,-1.0


<a name="sampling"></a>
# Permutation and Random Sampling

The `numpy.random.permutation` function is used to permute (randomly reorder) the values in a Series or the rows in a DataFrame.  

A Series can be directly permuted, but for a DataFrame, we have to permute the row or column indices and then sort the DataFrame accordingly

In [159]:
# Create a DataFrame
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [160]:
# Grab a Series from it
ser = df[0]
ser

0     0
1     7
2    14
3    21
4    28
Name: 0, dtype: int64

In [161]:
# Permute the values in the Series
np.random.permutation(ser)

array([28, 21, 14,  7,  0])

In [162]:
# Create a permutation using the number of rows in the DataFrame
rowSampler = np.random.permutation(5)
rowSampler

array([0, 4, 1, 2, 3])

In [163]:
# Sort the DataFrame using iloc and the sampler
df.iloc[rowSampler]

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27


In [164]:
# The take function can also be used
df.take(rowSampler)

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27


In [165]:
# Create a permutation for the columns
colSampler = np.random.permutation(7)
colSampler

array([4, 5, 2, 6, 3, 1, 0])

In [166]:
# Re-order the columns with iloc
df.iloc[:,colSampler]

,4,5,2,6,3,1,0
0,4,5,2,6,3,1,0
1,11,12,9,13,10,8,7
2,18,19,16,20,17,15,14
3,25,26,23,27,24,22,21
4,32,33,30,34,31,29,28


In [167]:
# Same thing with take
df.take(colSampler, axis="columns")

,4,5,2,6,3,1,0
0,4,5,2,6,3,1,0
1,11,12,9,13,10,8,7
2,18,19,16,20,17,15,14
3,25,26,23,27,24,22,21
4,32,33,30,34,31,29,28


Instead of permuting all of the rows/columns we can also take a random sample either with or without replacement.

In [168]:
# Random sample of rows
df.sample(n = 3)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
4,28,29,30,31,32,33,34


In [169]:
# Random sample of columns
df.sample(n = 3, axis="columns")

,6,3,2
0,6,3,2
1,13,10,9
2,20,17,16
3,27,24,23
4,34,31,30


In [170]:
# Random sample of a series, with replacement
ser.sample(n=10, replace=True)

2    14
1     7
3    21
3    21
1     7
4    28
4    28
4    28
2    14
0     0
Name: 0, dtype: int64

<a name="dummy"></a>
# Computing Indicator/Dummy Variables

Not too familiar with this type of transformation, but it is used for statistical modeling and machine learning applications.  Closes thing I can think of right now are DESEQ contrasts.  

Essentially, you might want to convert a categorical variable into a dummy/indicator matrix.  

This necessarily increases the dimension by one. So a single column of a DataFrame can be converted to a dummy matrix.

In [171]:
# Make data
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [172]:
# Dummy matrix
    # one column for each unique value in key
    # Row 0: 0, 1, 0
    # Row 1: 0, 1, 0
    # Row 2: 1, 0, 0
    # etc.
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [173]:
# True/False are returned by default
pd.get_dummies(df["key"])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [174]:
# Add a prefix to the newly-formed columns
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [175]:
# With the column prefixes, merging with the original data1 column makes more sense
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


It is a little more involved to create dummy variables for a DataFrame whose rows have multiple categories.  

In the example below, we can see that genres have multiple entires separated by `|`:

In [176]:
# Make a DataFrame of movies
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("../../datasets/movielens/movies.dat", sep="::",
                       header=None, names=mnames, engine="python")

# View first 10 rows (same as head)
movies.head(n=10)
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


`str.get_dummies` is a special method that can handle multi-group membership with a delimited string

In [177]:
genreDummies = movies["genres"].str.get_dummies("|")
genreDummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


In [178]:
# Merge with the original movies DF
movies_windic = movies.join(genreDummies.add_prefix("Genre_"))

In [179]:
# View merged data
movies_windic

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House (2000),Drama,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [180]:
# View just a single row as a Series
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

Another use of dummies is to bin first and then make dummies:

In [181]:
# Set seed
np.random.seed(12345)

In [182]:
# Array of data
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [183]:
# List of bins
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [184]:
# Get dummy of the bins.
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False
